# 设置数据文件存放根目录（不直接放在该项目中，该项目有github仓库，避免大内存数据影响代码推送）

In [ ]:
boot_file_adr = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST"

# 建立映射表

In [ ]:
import pandas as pd
import os

# ========================
# 路径配置
# ========================
# boot_file_adr = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST"
raw_csv_dir = os.path.join(boot_file_adr, "raw_csv")
save_dir = os.path.join(boot_file_adr, "mapping_table")
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "KAIST_vibration_mapping.csv")

# ========================
# 获取文件列表
# ========================
csv_files = sorted([f for f in os.listdir(raw_csv_dir) if f.endswith(".csv")])

# ========================
# 标签映射（Normal=1，不同转速/故障独立编号）
# ========================
mapping_rules = {
    "Normal": 1,
    "BPFI_03": 2,
    "BPFI_10": 3,
    "BPFI_30": 4,
    "BPFO_03": 5,
    "BPFO_10": 6,
    "BPFO_30": 7,
    "Misalign_01": 8,
    "Misalign_03": 9,
    "Misalign_05": 10,
    "Unbalance_0583mg": 11,
    "Unbalance_1169mg": 12,
    "Unbalance_1751mg": 13,
    "Unbalance_2239mg": 14,
    "Unbalance_3318mg": 15,
}

# ========================
# 生成映射表
# ========================
records = []
for f in csv_files:
    matched_label = None
    for key, label in mapping_rules.items():
        if key in f:
            matched_label = label
            break
    if matched_label is None:
        matched_label = -1
    records.append({
        "filename": f,
        "label": matched_label
    })

df = pd.DataFrame(records)
df.to_csv(save_path, index=False)

print(f"✅ 已生成映射表：{save_path}")
display(df)


# 数据混合

## 全类别混合，用于基本模型训练评估

### mixed data->csv

In [1]:
import os
import pandas as pd

# ===================== 1️⃣ 参数设置 =====================
raw_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/raw_csv"
mapping_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mapping_table/KAIST_vibration_mapping.csv"
save_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/allclass_CWT.csv"
used_index_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/used_indices.csv"

# ========== 你需要的每类抽取数量 ==========
sample_dict = {
    1: 50000,
    2: 50000,
    3: 50000,
    4: 50000,
    5: 50000,
    6: 50000,
    7: 50000,
    8: 50000,
    9: 50000,
    10: 50000,
    11: 50000,
    12: 50000,
    13: 0,
    14: 50000,
}

# ===================== 2️⃣ 读取映射表 =====================
mapping = pd.read_csv(mapping_path)
print(f"📘 映射表已读取，共 {len(mapping)} 条记录")

# ===================== 3️⃣ 加载已使用索引 =====================
if os.path.exists(used_index_path):
    used_indices = pd.read_csv(used_index_path)
else:
    used_indices = pd.DataFrame(columns=["source_file", "index", "label"])

# ===================== 4️⃣ 初始化输出文件 =====================
if os.path.exists(save_path):
    os.remove(save_path)
    print("⚙️ 已清空旧的 mixed_allclass.csv")
first_write = True

# ===================== 5️⃣ 实际抽样开始 =====================
# 遍历每一类
for label, target_n in sample_dict.items():

    print(f"\n==============================")
    print(f"🎯 开始处理类别 {label}，目标抽取 {target_n} 条")
    print(f"==============================")

    # 统计当前类别已抽取多少
    already_used = used_indices.loc[used_indices["label"] == label].shape[0]
    print(f"📊 当前类别 {label} 已有 {already_used} 条")

    if already_used >= target_n:
        print(f"✅ 类别 {label} 样本已足够（{already_used} >= {target_n}），跳过本类。")
        continue

    # 还需要抽的数
    remain = target_n - already_used
    print(f"🔍 还需 {remain} 条样本")

    # 遍历所有文件，属于该类的都尝试抽取
    for fname in os.listdir(raw_dir):

        if not fname.endswith(".csv"):
            continue
        if "2Nm" in fname:   # 排除 2Nm 文件
            continue

        # 找当前文件的标签
        entry = mapping.loc[mapping["filename"] == fname]
        if entry.empty:
            continue

        file_label = int(entry["label"].values[0])
        if file_label != label:
            continue

        file_path = os.path.join(raw_dir, fname)
        df = pd.read_csv(file_path)
        df["label"] = label
        df["source_file"] = fname

        # 当前文件未使用的样本
        used_idx_this_file = used_indices.loc[used_indices["source_file"] == fname, "index"].tolist()
        df_avail = df.loc[~df.index.isin(used_idx_this_file)]

        if df_avail.empty:
            print(f"⚠️ {fname} 可用数据为 0，跳过。")
            continue

        # 本文件可抽取数量
        take_n = min(remain, len(df_avail))

        sampled_df = df_avail.sample(n=take_n, random_state=42)

        # 写入输出文件
        sampled_df.to_csv(save_path, mode="a", header=first_write, index=False)
        first_write = False

        # 更新 used_indices
        new_used = pd.DataFrame({
            "source_file": fname,
            "index": sampled_df.index,
            "label": label
        })
        used_indices = pd.concat([used_indices, new_used], ignore_index=True)

        print(f"📦 从 {fname} 抽取 {take_n} 条。")

        remain -= take_n
        if remain <= 0:
            print(f"🎉 类别 {label} 已抽满 {target_n} 条。")
            break

# ===================== 6️⃣ 保存使用记录 =====================
used_indices.to_csv(used_index_path, index=False)
print("\n==============================")
print(f"🏁 所有类别抽样完成！")
print(f"📁 已保存：{save_path}")
print(f"📁 使用记录：{used_index_path}")
print("==============================")
# ===================== 7️⃣ 删除最终文件的最后一列 =====================
print("\n🧹 正在删除最终 CSV 的最后一列 ...")

df_final = pd.read_csv(save_path)

# 删除最后一列
last_col = df_final.columns[-1]
df_final = df_final.iloc[:, :-1]

# 覆盖保存
df_final.to_csv(save_path, index=False)

print(f"🗑 已删除最后一列：{last_col}")
print(f"📁 文件已更新：{save_path}")


📘 映射表已读取，共 15 条记录

🎯 开始处理类别 1，目标抽取 50000 条
📊 当前类别 1 已有 0 条
🔍 还需 50000 条样本
📦 从 0Nm_Normal.csv 抽取 50000 条。
🎉 类别 1 已抽满 50000 条。

🎯 开始处理类别 2，目标抽取 50000 条
📊 当前类别 2 已有 0 条
🔍 还需 50000 条样本
📦 从 0Nm_BPFI_03.csv 抽取 50000 条。
🎉 类别 2 已抽满 50000 条。

🎯 开始处理类别 3，目标抽取 50000 条
📊 当前类别 3 已有 0 条
🔍 还需 50000 条样本
📦 从 0Nm_BPFI_10.csv 抽取 50000 条。
🎉 类别 3 已抽满 50000 条。

🎯 开始处理类别 4，目标抽取 50000 条
📊 当前类别 4 已有 0 条
🔍 还需 50000 条样本
📦 从 0Nm_BPFI_30.csv 抽取 50000 条。
🎉 类别 4 已抽满 50000 条。

🎯 开始处理类别 5，目标抽取 50000 条
📊 当前类别 5 已有 0 条
🔍 还需 50000 条样本
📦 从 0Nm_BPFO_03.csv 抽取 50000 条。
🎉 类别 5 已抽满 50000 条。

🎯 开始处理类别 6，目标抽取 50000 条
📊 当前类别 6 已有 0 条
🔍 还需 50000 条样本
📦 从 0Nm_BPFO_10.csv 抽取 50000 条。
🎉 类别 6 已抽满 50000 条。

🎯 开始处理类别 7，目标抽取 50000 条
📊 当前类别 7 已有 0 条
🔍 还需 50000 条样本
📦 从 0Nm_BPFO_30.csv 抽取 50000 条。
🎉 类别 7 已抽满 50000 条。

🎯 开始处理类别 8，目标抽取 50000 条
📊 当前类别 8 已有 0 条
🔍 还需 50000 条样本
📦 从 0Nm_Misalign_01.csv 抽取 50000 条。
🎉 类别 8 已抽满 50000 条。

🎯 开始处理类别 9，目标抽取 50000 条
📊 当前类别 9 已有 0 条
🔍 还需 50000 条样本
📦 从 0Nm_Misalign_03.csv 抽取 50000 条。
🎉 类别 9 已抽满 50000 条。

🎯 

### 训练集、测试集、验证集划分，另存为新的csv文件

In [2]:
import os
import pandas as pd
import numpy as np

# ============= 1. 路径设置 =============
root_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT"
csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/allclass_CWT.csv"

print("加载数据：", csv_path)

train_dir = os.path.join(root_dir, "train")
val_dir = os.path.join(root_dir, "val")
test_dir = os.path.join(root_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# ============= 2. 读取数据 =============
df = pd.read_csv(csv_path)
print(f"数据加载成功，共 {len(df)} 条记录")

# 记录原始索引顺序
original_index = df.index.to_numpy()

# ============= 3. 设置划分比例 =============
train_ratio = 0.6
val_ratio = 0.1
test_ratio = 0.3
total = len(df)

# ============= 4. 随机抽取索引，不打乱顺序 =============
rng = np.random.default_rng(42)

# 随机采样，但不改变原始顺序
all_indices = np.arange(total)
rng.shuffle(all_indices)

train_end = int(total * train_ratio)
val_end = train_end + int(total * val_ratio)

train_idx = np.sort(all_indices[:train_end])
val_idx = np.sort(all_indices[train_end:val_end])
test_idx = np.sort(all_indices[val_end:])

# ============= 5. 保持原顺序抽取数据 =============
train_df = df.loc[train_idx].copy()
val_df = df.loc[val_idx].copy()
test_df = df.loc[test_idx].copy()

# # ============= 6. 删除最后一列（source_file） =============
# last_col = df.columns[-1]
# print(f"移除最后一列：{last_col}")

# train_df = train_df.iloc[:, :-1]
# val_df = val_df.iloc[:, :-1]
# test_df = test_df.iloc[:, :-1]

# ============= 7. 保存 CSV =============
train_df.to_csv(os.path.join(train_dir, "train.csv"), index=False)
val_df.to_csv(os.path.join(val_dir, "val.csv"), index=False)
test_df.to_csv(os.path.join(test_dir, "test.csv"), index=False)

print("所有数据集已保存（保持原顺序，已去除最后一列）。")

# ============= 8. 保存日志 =============
log_path = os.path.join(root_dir, "split_log.txt")
with open(log_path, "w") as f:
    f.write("KAIST all_class 数据集随机划分日志（保持原顺序+忽略最后一列）\n\n")
    f.write(f"总样本数: {len(df)}\n")
    f.write(f"Train: {len(train_df)}\n")
    f.write(f"Val:   {len(val_df)}\n")
    f.write(f"Test:  {len(test_df)}\n\n")
    f.write("划分比例:\n")
    f.write(f"train_ratio = {train_ratio}\n")
    f.write(f"val_ratio   = {val_ratio}\n")
    f.write(f"test_ratio  = {test_ratio}\n")

print("日志写入完成。")


加载数据： /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/allclass_CWT.csv
数据加载成功，共 650000 条记录
所有数据集已保存（保持原顺序，已去除最后一列）。
日志写入完成。


### CWT

In [3]:
import os
import pandas as pd
import numpy as np
import torch
import pywt   # ⭐ 必须安装 pywavelets

# ====================== 配置部分 ======================
paths = {
    "train": "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/train.csv",
    "val":   "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/val/val.csv",
    "test":  "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/test/test.csv",
}

# 滑窗参数
WINDOW = 512
STEP   = 256

# CWT 参数
WIDTHS = np.arange(1, 32)     # 小波尺度（你可以调整）
WAVELET = 'morl'              # Morlet 小波


# ====================== 处理函数 ======================
def process_csv(split_name, csv_path):

    print(f"\n==================== {split_name.upper()} ====================")
    print(f"📄 加载数据：{csv_path}")

    df = pd.read_csv(csv_path)
    sig = df[["vibration_ch1", "vibration_ch2", "vibration_ch3", "vibration_ch4"]].values      # shape (N,4)
    labels = df["label"].values
    N = len(df)

    print(f"总样本数：{N}")

    cwt_rows = []
    window_starts = []
    window_labels = []

    # =======================================
    # 滑动窗口
    # =======================================
    for start in range(0, N - WINDOW + 1, STEP):

        chunk = sig[start:start + WINDOW]         # (WINDOW, 4)
        chunk_labels = labels[start:start + WINDOW]

        # --- 窗口标签（众数） ---
        (u, c) = np.unique(chunk_labels, return_counts=True)
        win_label = u[np.argmax(c)]

        win_feat = []

        # =======================================
        # 4 通道 CWT
        # =======================================
        for ch in range(4):

            x = chunk[:, ch]                     # (WINDOW,)

            cwt_mat, _ = pywt.cwt(x, WIDTHS, WAVELET)
            cwt_abs = np.abs(cwt_mat)

            cwt_mean = cwt_abs.mean(axis=1)      # 对时间维求均值 → 长度 = len(WIDTHS)

            win_feat.extend(cwt_mean.tolist())

        # 保存窗口特征
        cwt_rows.append(win_feat)
        window_starts.append(start)
        window_labels.append(win_label)

    # =======================================
    # 构建输出 DataFrame
    # =======================================
    cwt_rows = np.array(cwt_rows)
    scale_dim = len(WIDTHS)

    print(f"窗口数：{len(cwt_rows)}")
    print(f"每窗口特征维度：{cwt_rows.shape[1]} (4 通道 × {scale_dim} scales)")

    # 构建列名
    columns = []
    for ch in range(4):
        for s in range(scale_dim):
            columns.append(f"ch{ch+1}_scale{s}")

    df_out = pd.DataFrame(cwt_rows, columns=columns)
    df_out.insert(0, "window_start", window_starts)
    df_out["label"] = window_labels

    # 保存路径
    save_path = os.path.join(os.path.dirname(csv_path), f"{split_name}_CWT.csv")
    df_out.to_csv(save_path, index=False)

    print(f"✅ 已保存：{save_path}")


# ====================== 处理 train/val/test ======================
for name, path in paths.items():
    process_csv(name, path)

print("\n🎉 处理完成！（train / val / test 均已生成新 CWT CSV）")



==================== TRAIN ====================
📄 加载数据：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/train.csv
总样本数：390000
窗口数：1522
每窗口特征维度：124 (4 通道 × 31 scales)
✅ 已保存：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/train_CWT.csv

==================== VAL ====================
📄 加载数据：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/val/val.csv
总样本数：65000
窗口数：252
每窗口特征维度：124 (4 通道 × 31 scales)
✅ 已保存：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/val/val_CWT.csv

==================== TEST ====================
📄 加载数据：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/test/test.csv
总样本数：195000
窗口数：760
每窗口特征维度：124 (4 通道 × 31 scales)
✅ 已保存：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/test/test_CWT.csv

🎉 处理完成！（train / val / test 均已生成新 CWT CSV）


### mixed_data->pyg

#### 时间邻点建图

##### CWT数据建图

In [4]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import build_local_temporal_graph   # ★ 你要用的函数
import pandas as pd
import os
import torch

# ================== 2️⃣ 根目录 ==================
root_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT"

splits = {
    "train": os.path.join(root_dir, "train/train_CWT.csv"),
    "val":   os.path.join(root_dir, "val/val_CWT.csv"),
    "test":  os.path.join(root_dir, "test/test_CWT.csv"),
}

# ================== 3️⃣ 建图参数可调 ==================
num_edges = 10     # 时间相邻 KNN 数（你自己定）

# ================== 4️⃣ 分别为三个集建图 ==================
for split_name, csv_path in splits.items():

    print(f"\n🚀 开始构建 {split_name} 图结构...")
    assert os.path.exists(csv_path), f"❌ CSV 文件不存在：{csv_path}"

    # === 保存路径：train/pyg/、val/pyg/、test/pyg/ ===
    pyg_dir = os.path.join(os.path.dirname(csv_path), "CWT_pyg")
    os.makedirs(pyg_dir, exist_ok=True)

    print(f"📁 pyg 保存目录: {pyg_dir}")

    # === 调用本地时序建图函数 ===
    nodes_csv, edges_csv, graph_pt = build_local_temporal_graph(
        csv_path=csv_path,
        save_dir=pyg_dir,
        num_edges=num_edges
    )

    print(f"   📄 nodes_csv: {nodes_csv}")
    print(f"   📄 edges_csv: {edges_csv}")
    print(f"   📦 graph_pt : {graph_pt}")

    print(f"✅ {split_name} 图构建完成。")

print("\n🎉 三个数据集 (train/val/test) 的图已经全部独立构建完毕！")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib

🚀 开始构建 train 图结构...
📁 pyg 保存目录: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/CWT_pyg
✅ 图结构构建完成，共 1522 个节点，15220 条无向边
📁 nodes.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/CWT_pyg/nodes.csv
📁 edges.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/CWT_pyg/edges.csv
📁 graph.pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/CWT_pyg/graph.pt
🧩 特征维度: 124 (已自动忽略首列与标签列)
   📄 nodes_csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/CWT_pyg/nodes.csv
   📄 edges_csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/CWT_pyg/edges.csv
   📦 graph_pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/CWT_pyg/graph.pt
✅ train 图构建完成。

🚀 开

In [5]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py

import torch
import os
import numpy as np

# ================== 2️⃣ 读取 train 图 ==================
train_graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/CWT_pyg/graph.pt"
assert os.path.exists(train_graph_path), f"❌ 文件不存在：{train_graph_path}"

print(f"📌 加载 train 图：{train_graph_path}")
data = torch.load(train_graph_path)

total_nodes = data.x.size(0)
print(f"节点数 = {total_nodes}")

# ================== 3️⃣ 设置比例 ==================
train_label_ratio = 0.3   # 30% 节点为 train_withlabel

num_withlabel = int(total_nodes * train_label_ratio)
num_nolabel = total_nodes - num_withlabel

print(f"👉 有标签节点：{num_withlabel}  | 无标签节点：{num_nolabel}")

# ================== 4️⃣ 随机生成 mask（保持可复现） ==================
rng = np.random.default_rng(42)

all_indices = np.arange(total_nodes)
rng.shuffle(all_indices)

withlabel_idx = all_indices[:num_withlabel]
nolabel_idx   = all_indices[num_withlabel:]

# 创建 mask
train_withlabel_mask = np.zeros(total_nodes, dtype=bool)
train_nolabel_mask   = np.zeros(total_nodes, dtype=bool)

train_withlabel_mask[withlabel_idx] = True
train_nolabel_mask[nolabel_idx] = True

# ================== 5️⃣ 添加到 Data 对象 ==================
data.train_withlabel_mask = torch.tensor(train_withlabel_mask)
data.train_nolabel_mask   = torch.tensor(train_nolabel_mask)

print("✅ 掩码添加完成：")
print(f"  train_withlabel_mask.sum() = {int(data.train_withlabel_mask.sum())}")
print(f"  train_nolabel_mask.sum()   = {int(data.train_nolabel_mask.sum())}")

# ================== 6️⃣ 保存新的图结构 ==================
save_path = os.path.join(
    os.path.dirname(train_graph_path),
    "train_graph_with_labelmask.pt"
)

torch.save(data, save_path)
print(f"🎉 新图已保存为：{save_path}")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
📌 加载 train 图：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/CWT_pyg/graph.pt
节点数 = 1522
👉 有标签节点：456  | 无标签节点：1066
✅ 掩码添加完成：
  train_withlabel_mask.sum() = 456
  train_nolabel_mask.sum()   = 1066
🎉 新图已保存为：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/CWT_pyg/train_graph_with_labelmask.pt


/tmp/ipykernel_107840/3474244181.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(train_graph_path)


#### 余弦相似度建图

In [ ]:
# # ================== 1️⃣ 导入路径与函数 ==================
# %run ../_init_path.py
# from build_pyg_data import build_similarity_knn_graph
# import pandas as pd
# import torch
# import os

# # ================== 2️⃣ 参数设置 ==================
# csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass_nofinal.csv"
# save_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn"

# # ================== 3️⃣ 调用建图函数 ==================
# print("🚀 开始构建基于余弦相似度的KNN图 ...")
# nodes_csv, edges_csv, graph_pt = build_similarity_knn_graph(
#     csv_path=csv_path,
#     save_dir=save_dir,
#     num_edges=10
# )
# print("\n✅ 图结构已生成：")
# print(f"📁 节点文件: {nodes_csv}")
# print(f"📁 边文件:   {edges_csv}")
# print(f"📁 图文件:   {graph_pt}")

# # ================== 4️⃣ 验证生成的图数据 ==================
# print("\n🧩 验证生成的 PyG 图数据结构 ...")

# # 读取 PyG 对象
# data = torch.load(graph_pt, weights_only=False)

# # === 基础信息 ===
# num_nodes = data.x.size(0)
# num_edges = data.edge_index.size(1)
# num_features = data.x.size(1)
# has_label = hasattr(data, "y") and data.y is not None

# print(f"📦 Data 对象结构: {data}")
# print(f"🧮 节点数: {num_nodes}")
# print(f"🧮 边数: {num_edges // 2}（无向边）")
# print(f"🧩 特征维度: {num_features}")
# print(f"🏷️  标签存在: {'✅ 是' if has_label else '❌ 否'}")

# # === 标签验证 ===
# if has_label:
#     y = data.y.cpu().numpy()
#     unique_labels = sorted(set(y))
#     print(f"🎯 标签类别数量: {len(unique_labels)} | 类别: {unique_labels[:10]}")
#     print(f"前5个标签: {y[:5]}")
# else:
#     print("⚠️ 未检测到 data.y，标签列可能缺失或未正确指定。")

# # === 节点/边文件验证 ===
# print("\n📊 验证节点和边 CSV 文件 ...")
# df_nodes = pd.read_csv(nodes_csv)
# df_edges = pd.read_csv(edges_csv)

# print(f"✅ 节点文件行数: {len(df_nodes)} | 与图节点数 {'一致 ✅' if len(df_nodes)==num_nodes else '不一致 ⚠️'}")
# print(f"✅ 边文件行数: {len(df_edges)} | 每两行对应一条无向边")

# if "source" in df_edges.columns and "target" in df_edges.columns:
#     unique_nodes = pd.concat([df_edges["source"], df_edges["target"]]).nunique()
#     print(f"🧮 边中涉及的唯一节点数: {unique_nodes}")
# else:
#     print("⚠️ 边文件缺少 'source' 或 'target' 列！")

# print("\n🎯 图数据验证完成。")


In [ ]:
# # ================== 1️⃣ 导入路径与函数 ==================
# %run ../_init_path.py
# from build_pyg_data import add_random_masks_with_label_split
# import pandas as pd
# import torch
# import os

# # ================== 2️⃣ 参数设置 ==================
# ratios = {"train": 0.6, "val": 0.2, "test": 0.2}

# graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_graph.pt"
# save_path  = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_arabic_graph_withmask.pt"

# # ================== 3️⃣ 生成掩码 ==================
# print("🚀 开始为图添加随机掩码 ...")
# new_graph_path = add_random_masks_with_label_split(
#     graph_path=graph_path,
#     save_path=save_path,     # 若想覆盖源文件则直接传 None
#     ratios=ratios,
#     train_label_ratio=0.4,   # 训练集中40%为有标签
# )
# print(f"\n✅ 掩码生成与保存完成：{new_graph_path}")

# # ================== 4️⃣ 验证生成结果 ==================
# print("\n🧩 验证生成的图结构与掩码 ...")

# # 读取 PyG 数据
# data = torch.load(new_graph_path, weights_only=False)

# # === 基本信息 ===
# print(f"\n📦 图数据结构: {data}")
# print(f"x shape: {data.x.shape}, y shape: {data.y.shape}")
# print(f"前5个标签: {data.y[:5].tolist()}")

# # === 掩码存在性检测 ===
# mask_attrs = ["train_mask", "val_mask", "test_mask", "train_withlabel_mask", "train_nolabel_mask"]
# print("\n📋 掩码存在性检查：")
# for attr in mask_attrs:
#     print(f"  {attr:<25}: {'✅ 存在' if hasattr(data, attr) else '❌ 不存在'}")

# # === 掩码统计 ===
# if hasattr(data, "train_mask"):
#     total_nodes = data.x.size(0)
#     n_train = int(data.train_mask.sum())
#     n_val   = int(data.val_mask.sum()) if hasattr(data, "val_mask") else 0
#     n_test  = int(data.test_mask.sum()) if hasattr(data, "test_mask") else 0
#     n_with  = int(data.train_withlabel_mask.sum()) if hasattr(data, "train_withlabel_mask") else 0
#     n_nol   = int(data.train_nolabel_mask.sum()) if hasattr(data, "train_nolabel_mask") else 0

#     print("\n📊 掩码数量验证：")
#     print(f"  train_mask         = {n_train}")
#     print(f"  val_mask           = {n_val}")
#     print(f"  test_mask          = {n_test}")
#     print(f"  └─ train_withlabel = {n_with}")
#     print(f"  └─ train_nolabel   = {n_nol}")
#     print(f"  合计节点数         = {total_nodes}")
#     if n_train + n_val + n_test == total_nodes:
#         print("✅ 掩码划分总数匹配节点总数")
#     else:
#         print("⚠️ 注意：掩码划分节点数与总数不完全匹配，可能存在舍入误差。")

# # === 标签统计 ===
# unique_labels = sorted(set(data.y.cpu().numpy().tolist()))
# print("\n🎯 标签分布检查：")
# print(f"  标签类别数: {len(unique_labels)}")
# print(f"  类别示例: {unique_labels[:10]}")

# print("\n🎯 掩码与标签验证完成。")
